In [60]:
from collections import Counter
from typing import List, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import Dataset



print("start")

class FlagsDataset(Dataset):
    def __init__(self, path: str) -> None:
        super().__init__()
        
        self._data, self._vocab, self._unique_flags = self._load_data(path)
        self._word2idx = {word: idx + 1 for idx, word in enumerate(self._vocab)}
        self._flag2idx = {flag: idx for idx, flag in enumerate(self._unique_flags)}
        
        self.vocab_size = len(self._vocab) + 1
        self.n_classes = len(self._unique_flags)
        
    def _load_data(self, path: str) -> List:
        vocab, unique_flags = Counter(), Counter()
        data = []
        file = open(path, 'r')
        for line in file:
            flags = (line[line.index('[') + 1:line.index(']')]).split(', ')
            flags = [self._filter(flag) for flag in flags]
            unique_flags += Counter(flags)

            file_paths = (line[line.index('] [') + 3:]).replace(']', '').split(', ')
            file_paths = [self._tokenize(self._filter(file_path)) for file_path in file_paths]
            file_paths = [item for sublist in file_paths for item in sublist]
            vocab += Counter(file_paths)

            data.append((file_paths, flags))

        vocab = sorted(vocab, key=vocab.get, reverse=True)
        unique_flags = sorted(unique_flags, key=unique_flags.get, reverse=True)

        return data, vocab, unique_flags
    
    def _tokenize(self, text: str) -> List:
        return text.split('/')[1:]
    
    def _filter(self, text: str) -> str:
        text = text.replace('\n', '')
        text = text.replace("'", '')
        return text
    
    def __len__(self, ) -> int:
        return len(self._data)
    
    def __getitem__(self, idx: int) -> Tuple[List, List]:
        paths, flags = self._data[idx]
        
        inputs = torch.tensor([self._word2idx[word] for word in paths], dtype=torch.int64)
        #inputs = F.pad(inputs, (0, self._max_len - len(inputs)), value=0)
        
        labels = torch.zeros(self.n_classes, dtype=torch.float)
        labels[[self._flag2idx[flag] for flag in flags]] = 1
        
        return inputs, labels

start


In [2]:
class FlagClassifier(nn.Module):
    def __init__(self, vocab_size, n_classes, emb_dim=64, hidden_dim=128) -> None:
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.emb_layer = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hidden_dim)
        
        self.linear = nn.Linear(hidden_dim, n_classes)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.emb_layer(x)
        x, _ = self.lstm(x)
        x = self.linear(x[-1])
        return x

In [3]:
def collate_fn(batch: List[Tuple[torch.Tensor, torch.Tensor]]) -> torch.Tensor:
    inputs = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    return nn.utils.rnn.pad_sequence(inputs, padding_value=0), torch.stack(labels, axis=0)

dataset = FlagsDataset("dataset.txt")
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, collate_fn=collate_fn, shuffle=True)

device = torch.device("cpu")
model = FlagClassifier(dataset.vocab_size, dataset.n_classes, dataset._max_len).to(device)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters())

In [4]:
epochs = 100

for epoch in range(epochs):
    epoch_loss = 0.
    for batch in tqdm(dataloader):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        
        logits = model(inputs)
        loss = loss_fn(logits, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
    print(f"Epoch {epoch+1} finished, train loss: {epoch_loss / len(dataloader)}")

100%|██████████| 43/43 [00:46<00:00,  1.08s/it]


Epoch 1 finished, train loss: 0.2637136592421421


100%|██████████| 43/43 [00:59<00:00,  1.38s/it]


Epoch 2 finished, train loss: 0.045643650775038916


100%|██████████| 43/43 [01:55<00:00,  2.68s/it]


...

Epoch 98 finished, train loss: 0.006103636258346743


100%|██████████| 43/43 [00:09<00:00,  4.48it/s]


Epoch 99 finished, train loss: 0.006045668840754864


100%|██████████| 43/43 [00:09<00:00,  4.73it/s]

Epoch 100 finished, train loss: 0.005998720902256494


In [6]:
model = model.cpu()

print((model(dataset[0][0]).sigmoid() > 0.5).nonzero())
print(dataset[0][1].nonzero())
print(model(dataset[0][0]))
print(dataset[0][0])




tensor([[ 7],
        [24]])
tensor([[ 7],
        [24]])
tensor([ -6.0549,  -1.7437,  -8.8945,  -6.5495, -12.1324,  -9.5775,  -9.8230,
          1.6025,  -7.1738, -11.0155,  -9.0088,  -1.8157,  -8.9323,  -5.6583,
         -4.3313,  -6.2582,  -4.8948,  -9.1755,  -4.7792,  -6.1798,  -4.2904,
         -7.3718,  -5.4732,  -6.3603,   1.0459,  -3.9689,  -7.9345,  -3.2641,
         -8.0317,  -4.6927, -12.7539,  -6.9042, -11.8692,  -6.0222,  -8.3910,
         -8.0081,  -7.9376,  -6.0219,  -5.7270,  -9.2602,  -4.6173,  -6.9237,
         -8.2855,  -5.9983,  -9.6159,  -8.6556,  -9.7714,  -9.9828,  -8.5212,
         -4.1035,  -4.6489,  -3.8393,  -8.9859,  -7.0772, -10.0190,  -8.1355,
         -8.1221,  -6.0019,  -5.7970,  -8.8273,  -9.7669,  -7.9812,  -7.7109,
        -10.7248, -10.5166, -10.0818,  -4.0407,  -1.6661,  -8.6470,  -7.8943,
         -9.7733,  -4.7770,  -7.8542,  -5.6136,  -6.8574,  -7.3624,  -9.2182,
         -8.2442,  -9.1187,  -7.1989,  -7.4427,  -5.8105,  -9.0060,  -4.0272,
      

In [13]:
idx2flag = {idx: flag for flag, idx in dataset._flag2idx.items()}
for t in (model(dataset[0][0]).sigmoid() > 0.5).nonzero().detach().cpu().numpy().tolist():
    print(idx2flag[t[0]])

clang
Interp


In [15]:
torch.save(model, "./flags_classification.pt")

In [64]:
model2 = torch.load("flags_classification.pt")
model2.eval()
dataset2 = FlagsDataset("dataset.txt")

def find_name(line):
    start = line.index('/')
    end = line.rindex(' ')
    return line[start:end]

def parser(name, result):
    file = open(name, 'r')
    name = ""
    read_file = 0
    for line in file:
        if line.find("diff --git") != -1 and read_file == 0:
            name = find_name(line)
            read_file = 1
            result.append(name)
        elif line.find("diff --git") != -1 and read_file == 1:
            name = ""
            read_file = 0
        elif line[0] == "+" and read_file == 1:
            pass
            #print(line, end= ' ')
    file.close()

lst = []
parser("super_risc.test", lst)

idx2flag = {idx: flag for flag, idx in dataset2._flag2idx.items()}

tensor = []

for path in lst:
    path = path.split("/")
    for elem in path:
        if elem in dataset2._vocab:
            tensor.append(dataset2._vocab.index(elem) + 1)




for t in (model2(torch.Tensor(tensor).int()).sigmoid() > 0.5).nonzero().detach().cpu().numpy().tolist():
    print(idx2flag[t[0]])